### Pobieranie modelu 
Link do manualnego pobrania: https://www.comet.ml/plutasnyy/model-registry/emd2-bert?search=1.0.0

Ważne: środowisko wykonawcze -> zmien typ środowiska -> GPU

In [1]:
!pip install comet-ml pytorch-lightning transformers

     |████████████████████████████████| 245kB 18.5MB/s 
     |████████████████████████████████| 686kB 57.1MB/s 
     |████████████████████████████████| 1.5MB 52.7MB/s 
     |████████████████████████████████| 204kB 59.2MB/s 
     |████████████████████████████████| 512kB 54.5MB/s 
     |████████████████████████████████| 829kB 55.1MB/s 
     |████████████████████████████████| 276kB 52.4MB/s 
     |████████████████████████████████| 102kB 15.4MB/s 
     |████████████████████████████████| 890kB 59.7MB/s 
     |████████████████████████████████| 2.9MB 56.1MB/s 
     |████████████████████████████████| 1.3MB 52.2MB/s 
     |████████████████████████████████| 296kB 61.7MB/s 
     |████████████████████████████████| 143kB 62.5MB/s 
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=114e1bd048604d4f4fc45fc56eaf8ce7535dbd852b8bb79873db1bb2e886105b
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wh

In [2]:
!wget https://www.comet.ml/api/registry/model/item/download?modelItemId=bJJfGgbQ4s266JvhOlv5KpGdF -O model.zip

--2021-01-10 20:16:33--  https://www.comet.ml/api/registry/model/item/download?modelItemId=bJJfGgbQ4s266JvhOlv5KpGdF
Resolving www.comet.ml (www.comet.ml)... 52.204.124.35, 34.196.199.220
Connecting to www.comet.ml (www.comet.ml)|52.204.124.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘model.zip’

model.zip               [        <=>         ] 386.82M   177MB/s    in 2.2s    

2021-01-10 20:17:02 (177 MB/s) - ‘model.zip’ saved [405611090]



In [3]:
!unzip model.zip

Archive:  model.zip
  inflating: epoch=00-val_loss=0.7696-f1_cum=0.5785.ckpt  


### Wczytanie modelu oraz wag

In [4]:
import pytorch_lightning as pl
from transformers import BertTokenizerFast, BertForSequenceClassification, BertConfig
import numpy as np

class LitModule(pl.LightningModule):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.model = BertForSequenceClassification(
            BertConfig(num_labels=5, output_attentions=False, output_hidden_states=False)
        )

    def forward(self, *args, **kwargs):
        return self.model(*args, **kwargs)

model = LitModule.load_from_checkpoint('epoch=00-val_loss=0.7696-f1_cum=0.5785.ckpt')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', do_lower_case=True)

comet_ml is installed but `COMET_API_KEY` is not set.


In [5]:
model.eval()
model.freeze()
model.cuda()

LitModule(
  (model): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
 

### Predict
Plik test.csv to podzbiór pliku z danymi dostarczonymi z zadaniem, u nas on został wykonany za pomocą skryptu: preprocess.ipynb

In [6]:
!wget https://raw.githubusercontent.com/plutasnyy/emd2/main/test.csv -O test.csv

--2021-01-10 20:17:31--  https://raw.githubusercontent.com/plutasnyy/emd2/main/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28977017 (28M) [text/plain]
Saving to: ‘test.csv’

test.csv            100%[===================>]  27.63M  --.-KB/s    in 0.1s    

2021-01-10 20:17:32 (194 MB/s) - ‘test.csv’ saved [28977017/28977017]



In [7]:
import pandas as pd
df = pd.read_csv("test.csv")
df = df[['reviewText', 'score']]
df['score'] = df['score'].astype(int)
df.head()

,reviewText,score
0,24 7 has several different themes of Mahjong a...,5
1,I like reading the secrets that people have bu...,3
2,Where's my perry is a fun. it's lots like wher...,5
3,"who loves to shine poo? I do! come on, will yo...",5
4,I give it five stars because it's fun and cool...,5


In [8]:
import torch
from tqdm import tqdm_notebook

y_true, y_pred = list(), list()
for i, row in tqdm_notebook(df.iterrows(), total=len(df)):
    encode = tokenizer(row['reviewText'], add_special_tokens=True, padding='max_length', max_length=128, truncation=True)
    item = {key: torch.tensor(val).long() for key, val in encode.items()}
    
    outputs = model(
        item['input_ids'].unsqueeze(0).cuda(),
        token_type_ids=None,
        attention_mask=item['attention_mask'].unsqueeze(0).cuda()
    )  
    predeicted = np.argmax(outputs.logits.cpu().numpy().squeeze(), axis=-1).astype(int)

    y_true.append(row['score']-1) # model przewiduje od 0-4
    y_pred.append(predeicted)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [9]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred, digits=3))

              precision    recall  f1-score   support

           0      0.680     0.761     0.718      6014
           1      0.362     0.427     0.392      3374
           2      0.476     0.496     0.486      6389
           3      0.475     0.506     0.490     11653
           4      0.843     0.773     0.806     28145

    accuracy                          0.663     55575
   macro avg      0.567     0.593     0.578     55575
weighted avg      0.677     0.663     0.668     55575

